# if run on colab for test

In [1]:
# !git clone https://github.com/H999/GeneticCNN-torch-test-with-base-model.git
# %cd GeneticCNN-torch/

# import

In [2]:
import torch
import random
import numpy as np
import torchvision
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
from tqdm.auto import tqdm
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

from lib.save_Individual import save_Individual
from lib.load_Individual import load_Individual
from lib.train_Individual import train_Individual

from lib.train_Individual import test

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 64

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.MNIST('data', train=True, download=True, transform=transform)
testset = datasets.MNIST('data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)


def length(x): return len(x) * x.batch_size


print("Total train examples: ", length(train_loader))
print("Total test examples: ", length(test_loader))

C:\Users\HH\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\HH\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\HH\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Total train examples:  60032
Total test examples:  10048


In [3]:
def set_seed(seed):
    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True


seed = 1234
set_seed(seed)

# class Individual

In [4]:
from geneticCNN.Individual import Individual

# Train & test model for Individual

## init model base

In [5]:
EPOCHS = 5

In [6]:
a = Individual((3, 5), ('111','1111111111'))
optimizer = torch.optim.SGD(a.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

## test model base without train

In [7]:
a.to(device)
test(a, device, test_loader, F.nll_loss, 1, True)

Test Epoch 1:   0%|          | 0/157 [00:00<?, ?it/s]C:\Users\HH\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
                                                               


Test Epoch 1: Average loss: 2.30390558, Accuracy: 933/10000 (9.3300%)



(0.0933, 2.303905583190918)

## train and test model base

In [8]:
if Path('model.pt').exists():
    a, optimizer, scheduler = load_Individual()
    print('load model from model.pt')
else:
    train_Individual(a, device, train_loader, test_loader, optimizer, F.nll_loss, scheduler, EPOCHS, True)
    save_Individual(a, optimizer, scheduler)
    print('save model to model.pt')

 20%|██        | 1/5 [01:43<06:52, 103.21s/it]


Test Epoch 0: Average loss: 0.08997493, Accuracy: 9732/10000 (97.3200%)



 40%|████      | 2/5 [03:26<05:09, 103.18s/it]


Test Epoch 1: Average loss: 0.06339527, Accuracy: 9837/10000 (98.3700%)



 60%|██████    | 3/5 [05:10<03:27, 103.64s/it]


Test Epoch 2: Average loss: 0.03867957, Accuracy: 9889/10000 (98.8900%)



 80%|████████  | 4/5 [06:52<01:42, 102.89s/it]


Test Epoch 3: Average loss: 0.03052328, Accuracy: 9914/10000 (99.1400%)




Test Epoch 4: Average loss: 0.03284622, Accuracy: 9913/10000 (99.1300%)

save model to model.pt


In [9]:
test(a, device, test_loader, F.nll_loss, 1, True)


Test Epoch 1: Average loss: 0.03284622, Accuracy: 9913/10000 (99.1300%)



(0.9913, 0.03284622202506289)

## test with different architecture model base weight

In [14]:
if Path('log.csv').exists():
    df = pd.read_csv('log.csv')
    print('load log.csv file to read/save test result')
else:
    df = pd.DataFrame(columns=['model tuple', 'gen', 'test result'])
    print('create log.csv file to save test result')

for i in range(3, 8):
    for j in range(1, 1024):

# for i in range(1023, 0, -1):
#     for j in range(7, 0, -1):

# for i in range(7, 0, -1):
#     for j in range(1023, 0, -1):
        print("\n===================\n")
        check = np.where(df['model tuple'] == str(("{0:03b}".format(i), "{0:010b}".format(j))))[0]
        if len(check) == 1:
            print(df.loc[check[0]]['gen'], '\n')
            print(df.loc[check[0]]['test result'], '\n')
        else:
            a.Stages.update_stages_state(("{0:03b}".format(i), "{0:010b}".format(j)))
            print(a.Stages.gen_model)
            df = df.append(
                {
                    'model tuple': a.Stages.gen,
                    'gen': a.Stages.gen_model,
                    'test result': (test(a, device, test_loader, F.nll_loss, 1, True))
                }, ignore_index=True)
            df.to_csv('log.csv', index=False)

load log.csv file to read/save test result


{'S_1': '0-11', 'S_2': '0-00-000-0001'}



Test Epoch 1: Average loss: 2.30433640, Accuracy: 923/10000 (9.2300%)



{'S_1': '0-11', 'S_2': '0-00-000-0010'}



Test Epoch 1: Average loss: 2.30658837, Accuracy: 592/10000 (5.9200%)



{'S_1': '0-11', 'S_2': '0-00-000-0011'}



Test Epoch 1: Average loss: 2.30380268, Accuracy: 939/10000 (9.3900%)



{'S_1': '0-11', 'S_2': '0-00-000-0100'}



Test Epoch 1: Average loss: 2.32056881, Accuracy: 794/10000 (7.9400%)



{'S_1': '0-11', 'S_2': '0-00-000-0101'}


...


Test Epoch 1: Average loss: 0.04135127, Accuracy: 9883/10000 (98.8300%)



{'S_1': '0-11', 'S_2': '0-10-110-1100'}



Test Epoch 1: Average loss: 0.05104280, Accuracy: 9852/10000 (98.5200%)



{'S_1': '0-11', 'S_2': '0-10-110-1101'}



Test Epoch 1: Average loss: 0.04177697, Accuracy: 9875/10000 (98.7500%)



{'S_1': '0-11', 'S_2': '0-10-110-1110'}



Test Epoch 1: Average loss: 0.04245679, Accuracy: 9870/10000 (98.7000%)



{'S_1': '0-11', 'S_2': '0-10-110-1111'}


KeyboardInterrupt: 